In [1]:
from tensorflow.keras import layers, Model, optimizers
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
import numpy as np #pip install numpy
import cv2 #pip install opencv-python
from sklearn.model_selection import train_test_split # pip install scikit-learn
import json

In [2]:
path_5 = "C:/Users/ttoft/AppData/Local/Programs/omniverse-launcher/images_56/"

In [3]:
l = []
images = []
for i in range(int(2000/5)):
    ys = []
    with open(path_5 + "camera_params_" + str(i * 5).zfill(4) + ".json", 'r') as f:
        dict = json.load(f)
        t_mat = dict["cameraViewTransform"]
        ys.append(t_mat[14])
    with open(path_5 + "camera_params_" + str(i * 5 + 4).zfill(4) + ".json", 'r') as f:
        dict = json.load(f)
        t_mat = dict["cameraViewTransform"]
        ys.append(t_mat[14])
    if ys[1]-ys[0] > 0:
        images.append([cv2.cvtColor(cv2.resize(cv2.imread(path_5 + "rgb_" + str(i * 5).zfill(4) + ".png"), (240, 240)), cv2.COLOR_BGR2RGB), cv2.cvtColor(cv2.resize(cv2.imread(path_5 + "rgb_" + str(i * 5 + 4).zfill(4) + ".png"), (240, 240)), cv2.COLOR_BGR2RGB)])
        l.append(ys[1]-ys[0])
x = np.asarray(images) / 255
y = np.asarray(l)
print(x.shape)
print(y.shape)



(395, 2, 240, 240, 3)
(395,)


In [4]:
print(y.max())
print(x.mean())

0.425413406203786
0.47749501665586436


In [ ]:
dset = tf.keras.utils.dataset_from

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, shuffle=True)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(355, 2, 240, 240, 3)
(355,)
(40, 2, 240, 240, 3)
(40,)


In [ ]:
in_1 = layers.Input(shape=(240, 240, 3))
x1 = layers.AvgPool2D(pool_size=(3, 3), strides=(3, 3))(in_1)
x1 = layers.Conv2D(16, 4, strides=(2, 2))(x1)
x1 = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x1)
x1 = layers.Conv2D(32, 5, padding='same')(x1)
x1 = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x1)
x1 = layers.Conv2D(64, 3, padding='same')(x1)
x1 = layers.Conv2D(64, 3, padding='same')(x1)
x1 = layers.Conv2D(64, 3, padding='same')(x1)
x1 = layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1))(x1)
x1 = layers.Flatten()(x1)
m1 = Model(inputs=in_1, outputs=x1)

in_2 = layers.Input(shape=(240, 240, 3))
x2 = layers.AvgPool2D(pool_size=(3, 3), strides=(3, 3))(in_2)
x2 = layers.Conv2D(16, 4, strides=(2, 2))(x2)
x2 = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x2)
x2 = layers.Conv2D(32, 5, padding='same')(x2)
x2 = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x2)
x2 = layers.Conv2D(64, 3, padding='same')(x2)
x2 = layers.Conv2D(64, 3, padding='same')(x2)
x2 = layers.Conv2D(64, 3, padding='same')(x2)
x2 = layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1))(x2)
x2 = layers.Flatten()(x2)
m2 = Model(inputs=in_2, outputs=x2)

input_1 = layers.Input(shape=(240, 240, 3))
input_2 = layers.Input(shape=(240, 240, 3))

output_1 = m1(input_1)
output_2 = m2(input_2)

concated = layers.Concatenate()([output_1, output_2])

x3 = layers.Dense(16, 'relu')(concated)
x3 = layers.Dense(16, 'relu')(x3)
x3 = layers.Dense(16, 'relu')(x3)
x3 = layers.Dense(16, 'relu')(x3)
x3 = layers.Dense(1, 'sigmoid')(x3)

final_model = Model(inputs=[input_1, input_2], outputs=x3)

In [ ]:
base_model_1 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_1.trainable = False
base_model_2 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3), name='res_2')
base_model_2.trainable = False

In [4]:
base_model_1 = tf.keras.models.load_model("object_recog_head.keras")
base_model_1.trainable = False
base_model_2 = tf.keras.models.load_model("object_recog_head.keras")
base_model_2.name = "res_2"
base_model_2.trainable = False

In [ ]:
input_1 = layers.Input(shape=(240, 240, 3))
input_2 = layers.Input(shape=(240, 240, 3))
input_3 = layers.Input(shape=(None, 6272))


#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(1, 'sigmoid')(x3)
regression_head = Model(inputs=input_3, outputs=x3)

output_1 = base_model_1(input_1)
output_2 = base_model_2(input_2)

concated = layers.Concatenate()([output_1, output_2])



final_model = Model(inputs=[input_1, input_2], outputs=regression_head(concated))

In [5]:
input_1 = layers.Input(shape=(240, 240, 3))
input_2 = layers.Input(shape=(240, 240, 3))

output_1 = base_model_1(input_1)
output_2 = base_model_2(input_2)

input_3 = layers.Input(shape=(7, 7, 128))

concated = layers.Concatenate()([output_1, output_2])



print(concated.shape)
x3 = layers.Flatten()(input_3)
#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
#x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(64, 'relu')(x3)
x3 = layers.Dense(1, 'sigmoid')(x3)
reg = Model(inputs=input_3, outputs=x3)
final_model = Model(inputs=[input_1, input_2], outputs=reg(concated))

(None, 7, 7, 128)


In [ ]:
final_model.compile(optimizer=optimizers.Adam(learning_rate=0.00000001), loss='mse')

In [7]:
final_model.summary()
base_model_1.summary()
reg.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 7, 7, 64)  │    105,968 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_2 (Functional)  │ (None, 7, 7, 64)  │    105,968 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7, 7, 128) │          0 │ functional_1[0][… │
│ (Concatenate)       │                   │            │ res_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 1)         │    414,017 │ concatenate[0][0] │
│ (Functional)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 625,953 (2.39 MB)

 Trainable params: 414,017 (1.58 MB)

 Non-trainable params: 211,936 (827.88 KB)

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       401,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 414,017 (1.58 MB)

 Trainable params: 414,017 (1.58 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
x_train = np.transpose(x_train, (1, 0, 2, 3, 4))
print(x_train.shape)

(2, 355, 240, 240, 3)


In [11]:
x_test = np.transpose(x_test, (1, 0, 2, 3, 4))
print(x_test.shape)

(2, 40, 240, 240, 3)


In [36]:
final_model.fit([x_train[0], x_train[1]], y_train, epochs=100, batch_size=8)

Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1.0256e-04
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.6589e-05
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.8024e-05
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0528e-04
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0303e-04
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.7151e-05
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.9654e-05
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.0112e-05
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0058e-04
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0782e-04
Epoch 11/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.4775e-05
Epoch 12/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.9073e-05
Epoch 13/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.7257e-05
Epoch 14/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.9134e-05
Epoch 15/100
45

In [37]:
final_model.evaluate([x_test[0], x_test[1]], y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 9.0298e-05 


9.131256229011342e-05

In [18]:
nr = 45
print(final_model([x_test[0][nr].reshape((1, 224, 224, 3)), x_test[1][nr].reshape((1, 224, 224, 3))])[0][0])
print(y_test[nr])

IndexError: index 45 is out of bounds for axis 0 with size 40

In [17]:
final_model.save("dist_from_2_img_snall_head_v2.keras")

In [ ]:
model_2 = tf.keras.models.load_model("dist_from_2_img_snall_head.keras")

In [ ]:
model_2.summary()

In [19]:
import time
import random

In [21]:
nr = 0
key = ' '
while key != 113:
    nr = random.randint(0, 39)
    to = time.time()
    cv2.imshow("hello", x_test[0][nr])
    print(final_model([x_test[0][nr].reshape((1, 240, 240, 3)), x_test[1][nr].reshape((1, 240, 240, 3))])[0][0])
    print(time.time() - to)
    print(y_test[nr])
    key = cv2.waitKey()
cv2.destroyAllWindows()

tf.Tensor(0.070048, shape=(), dtype=float32)
0.05402636528015137
0.0562354977219357
tf.Tensor(0.030944608, shape=(), dtype=float32)
0.022855043411254883
0.034818591169702096
tf.Tensor(0.10347651, shape=(), dtype=float32)
0.02591228485107422
0.11788339969091588
tf.Tensor(0.11633689, shape=(), dtype=float32)
0.02400040626525879
0.12286872766989454


In [22]:
import tf2onnx
import pickle

In [ ]:
tf.keras.models.Model.input_spec

In [ ]:
regression_head.summary()
regression_head.trainable=True

In [ ]:
test = tf.keras.models.load_model("object_recog_head.keras")

In [59]:
spec = (tf.TensorSpec((None, 7, 7, 128), tf.float32, name="input"),)
output_path = "reg_head.onnx"
model_proto = tf2onnx.convert.from_keras(reg, input_signature=spec, opset=13, output_path=output_path)

In [ ]:
x = np.concatenate([np.asarray(base_model_1([x_train[0]])), np.asarray(base_model_1([x_train[1]]))], axis=3)

c:\Users\ttoft\Masters_jupyter\.venv\lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_1
Received: inputs=['Tensor(shape=(355, 240, 240, 3))']
  warnings.warn(msg)


In [60]:
print(x.shape)
print(y_train.shape)

(355, 7, 7, 128)
(355,)


In [72]:
with open("reg_head.pickle", "wb") as f:
    pickle.dump(x, f)

In [73]:
with open("reg_head_2.pickle", "wb") as f:
    pickle.dump(y_train, f)

In [71]:
(x_t, y_t) = pickle.load(open("reg_head.pickle","rb"))

In [65]:
print(x_train[0].shape)
print(np.asarray(base_model_1([x_train[0]])).shape)

(355, 240, 240, 3)


c:\Users\ttoft\Masters_jupyter\.venv\lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_1
Received: inputs=['Tensor(shape=(355, 240, 240, 3))']
  warnings.warn(msg)


(355, 7, 7, 64)


In [66]:
with open("conv_head.pickle", "wb") as f:
    pickle.dump((x_train[0], np.asarray(base_model_1([x_train[0]]))), f)

In [53]:
y = np.asarray(reg(x))
print(y[1], y_train[1])

[0.12540956] 0.12138798991969857
